In [15]:
import keras
from PIL import Image
from keras import layers
import matplotlib.pyplot as plt
import numpy as np
import cv2
import tensorflow as tf
from PIL import Image
from keras.utils import to_categorical

In [16]:
image1 = [None] * 103
label1 = [None] * 103
image2 = [None] * 103
label2 = [None] * 103
#创建数据集列表
data_amount = 0

n = 0

#insulting
for i in range(1,100):
    #
    data_place_one = './data_training_20240213/insulting/' + str(i) + '.jpg'
    #print(data_place_one)
    img_2 = cv2.imread(data_place_one)
    if img_2 is not None:
        n += 1
        data_amount += 1
        image1[i-1] = np.array(img_2)
        label1[i-1] = 1
    else:
        pass
        #print("Failed to load image. or not exist")

#uninsulting
for i in range(1,100):
    data_place_one = './data_training_20240213/uninsulting/' + str(i) + '.jpg'
    #print(data_place_one)
    img_2 = cv2.imread(data_place_one)
    if img_2 is not None:
        n += 1
        data_amount += 1
        image2[i-1] = np.array(img_2)
        label2[i-1] = 0
    else:
        pass
        #print("Failed to load image. or not exist")
#print(img)

label = label1+label2
image = image1+image2

#删除空值
label = [x for x in label if x != None]
image = [x for x in image if np.any(x)]

test_image = image[80:120]
test_label = label[80:120]
image = image[:80]+image[120:]
label = label[:80]+label[120:]


In [17]:
label = np.array(label)
image = np.array(image)
test_label = np.array(test_label)
test_image = np.array(test_image)
#print(train_label.shape)

In [34]:
model = keras.Sequential([
    layers.Flatten(input_shape=(128, 128, 3)), 
    layers.Dense(16,activation="relu"),
    layers.Dense(10,activation="relu"),
    layers.Dense(2,activation="sigmoid")
])
model.compile(optimizer='rmsprop',
              loss = 'binary_crossentropy',
              metrics=['accuracy'])


In [35]:
# 将标签数据进行one-hot编码
label_encoded = to_categorical(label, num_classes=2)

In [47]:
history = model.fit(image,
        label_encoded,
        epochs=10,
        batch_size=100,
        )

Epoch 1/10
2/2 [==============================] - 0s 13ms/step - loss: 0.8544 - accuracy: 0.9747
Epoch 2/10
2/2 [==============================] - 0s 12ms/step - loss: 0.9879 - accuracy: 0.9684
Epoch 3/10
2/2 [==============================] - 0s 12ms/step - loss: 0.9620 - accuracy: 0.9747
Epoch 4/10
2/2 [==============================] - 0s 13ms/step - loss: 0.8164 - accuracy: 0.9810
Epoch 5/10
2/2 [==============================] - 0s 17ms/step - loss: 3.3382 - accuracy: 0.9304
Epoch 6/10
2/2 [==============================] - 0s 19ms/step - loss: 4.9106 - accuracy: 0.9367
Epoch 7/10
2/2 [==============================] - 0s 8ms/step - loss: 175.7539 - accuracy: 0.5190
Epoch 8/10
2/2 [==============================] - 0s 14ms/step - loss: 108.4617 - accuracy: 0.5696
Epoch 9/10
2/2 [==============================] - 0s 12ms/step - loss: 0.8100 - accuracy: 0.9747
Epoch 10/10
2/2 [==============================] - 0s 9ms/step - loss: 1.0436 - accuracy: 0.9747


In [48]:
# 将标签数据进行one-hot编码
test_label_encoded = to_categorical(test_label, num_classes=2)

In [49]:
model.evaluate(test_image,test_label_encoded)
y = np.argmax(model.predict(test_image[:]),axis=1)
print('原始：',y)
print('结果：',test_label)

2/2 [==============================] - 0s 3ms/step
原始： [0 0 0 0 0 1 1 1 0 1 0 0 1 1 1 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1
 0 1 0]
结果： [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0]
